In [1]:
from utils import *

In [2]:
import numpy as np
import pandas as pd

from collections import Counter

In [3]:
roots = {'history' : './data/百度题库/高中_历史/origin/', 
         'geology' : './data/百度题库/高中_地理/origin/',
         'politics' : './data/百度题库/高中_政治/origin/',
         'biology' : './data/百度题库/高中_生物/origin/'}

In [4]:
jieba.load_userdict('./stopwords/Special_words.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/09/k_9rj22d0dgbjd8832nhvlbh0000gn/T/jieba.cache
Loading model cost 0.569 seconds.
Prefix dict has been built succesfully.


In [5]:
def read_files(root):
    '''
    This function reads in all csv files lies directly under the root directory
    
    Returns the file directories as well as class names (file names)
    '''
    file_names = os.listdir(root)
    file_names = [name for name in file_names if name.endswith('csv')]
    classes = [name.split('.')[0] for name in file_names]
    file_names = [root + name for name in file_names]
    datasets = [pd.read_csv(name) for name in file_names]
    return datasets, classes

In [6]:
remove = "[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目|排除|选项|知识点"
def clean_sentence(line):
    '''
    This function cleans the context
    '''
    line = re.sub(remove, '', line)
    tokens = jieba.cut(line, cut_all=False)
    tokens = [token for token in tokens if token not in stopwords]
    return " ".join(tokens)

In [7]:
def clean_line(line):
    part1, part2 = line.split('题型', 1) # part 1 is 题目
    part2, part3 = part2.split('解析', 1) # part 2 is abanddoned
    part3 = part3.split('解析')[1]
    try:
        part3, part4 = part3.split('知识点', 1) # part 3 is 解析, part 4 is 知识点
    except ValueError:
        part4 = ''
    result = []
    for line in [part1, part3, part4]:
        result.append(clean_sentence(line))
    return result

In [8]:
def build_dataset(root):
    
    datasets, classes = read_files(root)
    
    for dataset, label in zip(datasets, classes):
        dataset['item'] = dataset['item'].apply(lambda x : clean_line(x))
        dataset['question'] = dataset['item'].apply(lambda x : x[0]).apply(lambda x : x.split())
        dataset['solution'] = dataset['item'].apply(lambda x : x[1]).apply(lambda x : x.split())
        dataset['keypoints'] = dataset['item'].apply(lambda x : x[2]).apply(lambda x : x.split())
        dataset['item'] = dataset['item'].apply(lambda x : ' '.join(x)).apply(lambda x : x.split())
        dataset['label'] = label
    
    dataset = pd.concat(datasets, ignore_index = True)
    dataset = dataset[['item', 'question', 'solution', 'keypoints', 'label']]
        
    return dataset

In [9]:
data = build_dataset(roots['politics'])

In [10]:
data.head()

,item,question,solution,keypoints,label
0,"[年, 政府, 工作, 报告, 提出, 缩小, 收入, 分配, 差距, 使, 发展, 成果,...","[年, 政府, 工作, 报告, 提出, 缩小, 收入, 分配, 差距, 使, 发展, 成果,...",[无],"[社会主义, 市场经济, 伦理, 要求, 劳动就业, 守法, 经营]",公民道德与伦理常识
1,"[做, 蛋糕, 分, 蛋糕, 经济社会, 面临, 最, 基本, 问题, 既要, 蛋糕, 做,...","[做, 蛋糕, 分, 蛋糕, 经济社会, 面临, 最, 基本, 问题, 既要, 蛋糕, 做,...",[无],"[社会主义, 市场经济, 伦理, 要求]",公民道德与伦理常识
2,"[最近, 常有, 手机用户, 收到, 老朋友, 名义, 发来, 短信, 短信, 极易, 引诱...","[最近, 常有, 手机用户, 收到, 老朋友, 名义, 发来, 短信, 短信, 极易, 引诱...",[无],"[社会主义, 市场经济, 伦理, 要求, 劳动就业, 守法, 经营]",公民道德与伦理常识
3,"[家庭, 人生, 第一, 课堂, 父母, 子女, 第一任, 教师, 家庭教育, 子女, 健康...","[家庭, 人生, 第一, 课堂, 父母, 子女, 第一任, 教师, 家庭教育, 子女, 健康...",[],"[公民, 道德, 生活]",公民道德与伦理常识
4,"[社会主义, 市场经济, 指, 市场, 国家, 下, 资源配置, 作用, 经济, 社会主义,...","[社会主义, 市场经济, 指, 市场, 国家, 下, 资源配置, 作用, 经济, 社会主义,...",[无],"[社会主义, 市场经济, 伦理, 要求, 劳动就业, 守法, 经营]",公民道德与伦理常识


In [11]:
counter = Counter()
for text in data['item']:
    counter.update(text)

In [12]:
counter.most_common()

[('社会主义', 4088),
 ('文化', 3915),
 ('发展', 3257),
 ('经济', 2555),
 ('市场经济', 2344),
 ('要求', 2206),
 ('中国', 1995),
 ('社会', 1914),
 ('伦理', 1786),
 ('中', 1774),
 ('我国', 1730),
 ('无', 1676),
 ('建设', 1470),
 ('人民', 1390),
 ('国家', 1385),
 ('生活', 1278),
 ('问题', 1263),
 ('市场', 1238),
 ('政府', 1150),
 ('年', 1142),
 ('收入', 1122),
 ('分配', 1057),
 ('经营', 996),
 ('运用', 958),
 ('不', 955),
 ('提高', 954),
 ('说明', 920),
 ('守法', 852),
 ('劳动就业', 838),
 ('分', 830),
 ('分析', 811),
 ('中国共产党', 808),
 ('坚持', 802),
 ('企业', 793),
 ('核心', 754),
 ('公平', 752),
 ('实现', 747),
 ('新', 711),
 ('加强', 704),
 ('上', 693),
 ('方式', 692),
 ('人', 685),
 ('科学', 683),
 ('作用', 674),
 ('主要', 660),
 ('特色', 650),
 ('道德', 646),
 ('经济学', 621),
 ('基本', 616),
 ('促进', 616),
 ('关系', 604),
 ('生产', 597),
 ('体现', 583),
 ('思维', 577),
 ('有利于', 567),
 ('都', 564),
 ('创新', 553),
 ('增长', 547),
 ('选择', 544),
 ('需要', 544),
 ('党', 535),
 ('理论', 533),
 ('年月日', 528),
 ('精神', 528),
 ('改革', 518),
 ('重要', 506),
 ('执政', 506),
 ('二', 502),
 ('必须', 496),
 ('影响', 489